In [1]:
!nvidia-smi

Thu Apr 28 23:14:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
!rm -r Fake-Face-Classification
!git clone https://github.com/suiboli314/Fake-Face-Classification.git
!pip install -r Fake-Face-Classification/requirements.txt

Cloning into 'Fake-Face-Classification'...
remote: Enumerating objects: 1449, done.
remote: Counting objects: 100% (1449/1449), done.
remote: Compressing objects: 100% (1413/1413), done.
remote: Total 1449 (delta 47), reused 1436 (delta 34), pack-reused 0
Receiving objects: 100% (1449/1449), 26.14 MiB | 31.63 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [3]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    dataset = "/content/drive/MyDrive/kaggle_df_dataset"
except:
    print("Note: not using Google CoLab")
    COLAB = False

!{dataset}

Mounted at /content/drive
Note: using Google CoLab
/bin/bash: /content/drive/MyDrive/kaggle_df_dataset: No such file or directory


In [15]:
%cd Fake-Face-Classification  
!git pull 
!git checkout dev-stylegan3
%cd ..

/content/Fake-Face-Classification
Already up to date.
Branch 'dev-stylegan3' set up to track remote branch 'dev-stylegan3' from 'origin'.
Switched to a new branch 'dev-stylegan3'
/content


In [6]:
import cv2
from google.colab.patches import cv2_imshow
from os import walk

size = (200,200)

def resize_all(down_size, path):
  filenames = next(walk(path), (None, None, []))[2]  # [] if no file
  for f in filenames:
    dir = "{}/{}".format(path, f)
    image = cv2.imread(dir)
    if image is not None:
      image = cv2.resize(image, down_size)
      cv2.imwrite(dir, image)

resize_all(size, "/content/Fake-Face-Classification/fake_real-faces/fake")
resize_all(size, "/content/Fake-Face-Classification/fake_real-faces/real")
print("done")

done


In [11]:
%matplotlib inline
save_dir = "/content/drive/MyDrive/model"
model_id = "test3"

In [9]:
!python Fake-Face-Classification/main.py --n_epochs=50 --save_dir={save_dir} --model_id={model_id} --img_size={size[0]}

Traceback (most recent call last):
  File "Fake-Face-Classification/main.py", line 114, in <module>
    train_torch()
  File "Fake-Face-Classification/main.py", line 101, in train_torch
    split_size=0.3, augment=True)
  File "/content/Fake-Face-Classification/datasets/dataset_torch.py", line 35, in pytorch_preprocess
    train_data = datasets.ImageFolder(dataset_dir + '/train', transform=train_transforms)
  File "/usr/local/lib/python3.7/dist-packages/torchvision/datasets/folder.py", line 316, in __init__
    is_valid_file=is_valid_file,
  File "/usr/local/lib/python3.7/dist-packages/torchvision/datasets/folder.py", line 145, in __init__
    classes, class_to_idx = self.find_classes(self.root)
  File "/usr/local/lib/python3.7/dist-packages/torchvision/datasets/folder.py", line 219, in find_classes
    return find_classes(directory)
  File "/usr/local/lib/python3.7/dist-packages/torchvision/datasets/folder.py", line 41, in find_classes
    classes = sorted(entry.name for entry in os.s

In [8]:
# plot if no plot shown
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd

def plot(history):
    plt.figure()
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epochs')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    plt.figure()
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

history = pd.read_csv(join(save_dir,"log_{}.csv".format(model_id)))
plot(history)

Output hidden; open in https://colab.research.google.com to view.

In [14]:
save_dir

'/content/drive/MyDrive/model'

In [17]:
!python Fake-Face-Classification/evaluation.py --save_dir={save_dir} --model_id={model_id} --img_size={size[0]}

Accuracy: 0.9599999785423279
Loss: 0.3197262432295531
